In [1]:
import cv2
import torch
import time
import numpy as np
import time
import datetime
from time import gmtime, strftime

In [2]:
model_type = "MiDaS_small"

'''

For higher accuracy/slower time:

model_type = "DPT_Hybrid"

For highest accuracy/slowest time:

model_type = "DPT_Large"

'''

midas = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

Using cache found in /Users/alifakhry/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/alifakhry/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


MidasNet_small(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2dSameExport(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU6(inplace=True)
          (se): Identity()
          (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(144,

In [3]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

Using cache found in /Users/alifakhry/.cache/torch/hub/intel-isl_MiDaS_master


In [4]:
def print_info(depth_map):
    img_reverted= cv2.bitwise_not(depth_map)
    new_img = img_reverted / 255.0 
    vertical_half = len(new_img)//2
    horizontal_half = len(new_img[0])//2
    
    print("*************")
    print("FIRST ROW,  MIDDLE COL", new_img[0][horizontal_half])
    print("FIRST ROW,  FIRST COL", new_img[0][0])
    print("MIDDLE ROW, LAST COL", new_img[vertical_half][len(new_img[0])-1])
    print("MIDDLE ROW, MIDDLE COL", new_img[vertical_half][horizontal_half])
    print("MIDDLE ROW, FIRST COL", new_img[vertical_half][0])
    print("*************")

In [5]:
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform
    
cap = cv2.VideoCapture(0)
    
while cap.isOpened():
    success, img = cap.read()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_batch = transform(img).to(device)
    
    with torch.no_grad():
        
        prediction = midas(input_batch)
        
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size = img.shape[:2],
            mode = "bicubic",
            align_corners = False,
        ).squeeze()
        
    depth_map = prediction.cpu().numpy()

    depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type = cv2.NORM_MINMAX, dtype=cv2.CV_64F)
    
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
    depth_map = (depth_map * 255).astype(np.uint8)
    depth_map = cv2.applyColorMap(depth_map, cv2.COLORMAP_BONE)

    cv2.putText(img, f'TIME (UTC): {str(strftime("%Y-%m-%d %H:%M:%S", gmtime()))}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow('Image', img)
    cv2.imshow('Depth Map', depth_map)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break
        
    depth_map = cv2.cvtColor(depth_map, cv2.COLOR_BGR2GRAY)
    
    print_info(depth_map)
    
cap.release()

*************
FIRST ROW,  MIDDLE COL 0.01568627450980392
FIRST ROW,  FIRST COL 0.5411764705882353
MIDDLE ROW, LAST COL 0.9058823529411765
MIDDLE ROW, MIDDLE COL 0.17254901960784313
MIDDLE ROW, FIRST COL 0.7294117647058823
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.9058823529411765
MIDDLE ROW, MIDDLE COL 0.043137254901960784
MIDDLE ROW, FIRST COL 0.9254901960784314
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.984313725490196
MIDDLE ROW, LAST COL 0.9411764705882353
MIDDLE ROW, MIDDLE COL 0.03529411764705882
MIDDLE ROW, FIRST COL 0.9137254901960784
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9882352941176471
MIDDLE ROW, LAST COL 0.8862745098039215
MIDDLE ROW, MIDDLE COL 0.03529411764705882
MIDDLE ROW, FIRST COL 0.9137254901960784
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098

*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.9019607843137255
MIDDLE ROW, MIDDLE COL 0.0392156862745098
MIDDLE ROW, FIRST COL 0.8745098039215686
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9882352941176471
MIDDLE ROW, LAST COL 0.9254901960784314
MIDDLE ROW, MIDDLE COL 0.03529411764705882
MIDDLE ROW, FIRST COL 0.8941176470588236
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.9215686274509803
MIDDLE ROW, MIDDLE COL 0.0392156862745098
MIDDLE ROW, FIRST COL 0.8862745098039215
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.8627450980392157
MIDDLE ROW, MIDDLE COL 0.0392156862745098
MIDDLE ROW, FIRST COL 0.8627450980392157
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FI

*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.9098039215686274
MIDDLE ROW, MIDDLE COL 0.047058823529411764
MIDDLE ROW, FIRST COL 0.8941176470588236
*************
*************
FIRST ROW,  MIDDLE COL 0.011764705882352941
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.8862745098039215
MIDDLE ROW, MIDDLE COL 0.050980392156862744
MIDDLE ROW, FIRST COL 0.8745098039215686
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.8941176470588236
MIDDLE ROW, MIDDLE COL 0.043137254901960784
MIDDLE ROW, FIRST COL 0.8823529411764706
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.9254901960784314
MIDDLE ROW, MIDDLE COL 0.050980392156862744
MIDDLE ROW, FIRST COL 0.9019607843137255
*************
*************
FIRST ROW,  MIDDLE COL 0.003921568627

*************
FIRST ROW,  MIDDLE COL 0.10588235294117647
FIRST ROW,  FIRST COL 0.9803921568627451
MIDDLE ROW, LAST COL 0.9607843137254902
MIDDLE ROW, MIDDLE COL 0.13333333333333333
MIDDLE ROW, FIRST COL 0.8745098039215686
*************
*************
FIRST ROW,  MIDDLE COL 0.047058823529411764
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.9529411764705882
MIDDLE ROW, MIDDLE COL 0.06666666666666667
MIDDLE ROW, FIRST COL 0.8666666666666667
*************
*************
FIRST ROW,  MIDDLE COL 0.058823529411764705
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.9294117647058824
MIDDLE ROW, MIDDLE COL 0.0784313725490196
MIDDLE ROW, FIRST COL 0.8509803921568627
*************
*************
FIRST ROW,  MIDDLE COL 0.12549019607843137
FIRST ROW,  FIRST COL 0.9921568627450981
MIDDLE ROW, LAST COL 0.9254901960784314
MIDDLE ROW, MIDDLE COL 0.1411764705882353
MIDDLE ROW, FIRST COL 0.8588235294117647
*************
*************
FIRST ROW,  MIDDLE COL 0.07450980392156863

*************
FIRST ROW,  MIDDLE COL 0.011764705882352941
FIRST ROW,  FIRST COL 0.9725490196078431
MIDDLE ROW, LAST COL 0.9490196078431372
MIDDLE ROW, MIDDLE COL 0.06274509803921569
MIDDLE ROW, FIRST COL 0.8823529411764706
*************
*************
FIRST ROW,  MIDDLE COL 0.0196078431372549
FIRST ROW,  FIRST COL 0.9607843137254902
MIDDLE ROW, LAST COL 0.9019607843137255
MIDDLE ROW, MIDDLE COL 0.0392156862745098
MIDDLE ROW, FIRST COL 0.8470588235294118
*************
*************
FIRST ROW,  MIDDLE COL 0.01568627450980392
FIRST ROW,  FIRST COL 0.9647058823529412
MIDDLE ROW, LAST COL 0.8666666666666667
MIDDLE ROW, MIDDLE COL 0.0392156862745098
MIDDLE ROW, FIRST COL 0.8117647058823529
*************
*************
FIRST ROW,  MIDDLE COL 0.00784313725490196
FIRST ROW,  FIRST COL 0.9450980392156862
MIDDLE ROW, LAST COL 0.8901960784313725
MIDDLE ROW, MIDDLE COL 0.03529411764705882
MIDDLE ROW, FIRST COL 0.8117647058823529
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
F

*************
FIRST ROW,  MIDDLE COL 0.0196078431372549
FIRST ROW,  FIRST COL 0.9529411764705882
MIDDLE ROW, LAST COL 0.9372549019607843
MIDDLE ROW, MIDDLE COL 0.043137254901960784
MIDDLE ROW, FIRST COL 0.8352941176470589
*************
*************
FIRST ROW,  MIDDLE COL 0.011764705882352941
FIRST ROW,  FIRST COL 0.9254901960784314
MIDDLE ROW, LAST COL 0.9647058823529412
MIDDLE ROW, MIDDLE COL 0.043137254901960784
MIDDLE ROW, FIRST COL 0.8274509803921568
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.984313725490196
MIDDLE ROW, LAST COL 0.9490196078431372
MIDDLE ROW, MIDDLE COL 0.03529411764705882
MIDDLE ROW, FIRST COL 0.8823529411764706
*************
*************
FIRST ROW,  MIDDLE COL 0.011764705882352941
FIRST ROW,  FIRST COL 0.984313725490196
MIDDLE ROW, LAST COL 0.9529411764705882
MIDDLE ROW, MIDDLE COL 0.0392156862745098
MIDDLE ROW, FIRST COL 0.8823529411764706
*************
*************
FIRST ROW,  MIDDLE COL 0.01568627450980392

*************
FIRST ROW,  MIDDLE COL 0.011764705882352941
FIRST ROW,  FIRST COL 0.984313725490196
MIDDLE ROW, LAST COL 0.9137254901960784
MIDDLE ROW, MIDDLE COL 0.0392156862745098
MIDDLE ROW, FIRST COL 0.9019607843137255
*************
*************
FIRST ROW,  MIDDLE COL 0.011764705882352941
FIRST ROW,  FIRST COL 0.984313725490196
MIDDLE ROW, LAST COL 0.9372549019607843
MIDDLE ROW, MIDDLE COL 0.050980392156862744
MIDDLE ROW, FIRST COL 0.8823529411764706
*************
*************
FIRST ROW,  MIDDLE COL 0.01568627450980392
FIRST ROW,  FIRST COL 0.984313725490196
MIDDLE ROW, LAST COL 0.9137254901960784
MIDDLE ROW, MIDDLE COL 0.050980392156862744
MIDDLE ROW, FIRST COL 0.8627450980392157
*************
*************
FIRST ROW,  MIDDLE COL 0.00392156862745098
FIRST ROW,  FIRST COL 0.9725490196078431
MIDDLE ROW, LAST COL 0.9019607843137255
MIDDLE ROW, MIDDLE COL 0.03529411764705882
MIDDLE ROW, FIRST COL 0.8745098039215686
*************
*************
FIRST ROW,  MIDDLE COL 0.00784313725490196

KeyboardInterrupt: 